In [ ]:
# Check CUDA version > 9.0
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


# Setup environment

In [ ]:
%%script bash

# Remove `sample_data` & create our folders
rm -rf sample_data
mkdir output
mkdir input

## Curl provided files

In [ ]:
%%script bash

cd input/

# save files in `/input`
curl https://raw.githubusercontent.com/christiaua/ECSE420/master/Lab3/files/input1.raw > input1.raw
curl https://raw.githubusercontent.com/christiaua/ECSE420/master/Lab3/files/input2.raw > input2.raw
curl https://raw.githubusercontent.com/christiaua/ECSE420/master/Lab3/files/input3.raw > input3.raw
curl https://raw.githubusercontent.com/christiaua/ECSE420/master/Lab3/files/input4.raw > input4.raw
curl https://raw.githubusercontent.com/christiaua/ECSE420/master/Lab3/files/sol_nextLevelNodes.raw > sol_nextLevelNodes.raw
curl https://raw.githubusercontent.com/christiaua/ECSE420/master/Lab3/files/sol_nodeOutput.raw > sol_nodeOutput.raw


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1160k  100 1160k    0     0  3171k      0 --:--:-- --:--:-- --:--:-- 3162k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  282k  100  282k    0     0   520k      0 --:--:-- --:--:-- --:--:--  520k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1748k  100 1748k    0     0  4027k      0 --:--:-- --:--:-- --:--:-- 4018k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57858  100 57858    0     0   163k      0 --:--:-- --:--:-- --:--:--  163k
  % Total    % Received % Xferd  Average Speed   Tim

## Write provided files

### Write read_input.c

In [ ]:
%%writefile read_input.c

#include <stdio.h>
#include <stdlib.h>

int read_input_one_two_four(int **input1, char* filepath){
 FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));

    int temp1;

    while (fscanf(fp, "%d", &temp1) == 1) {
        (*input1)[counter] = temp1;

        counter++;
    }

    fclose(fp);
    return len;
    



}
int read_input_three(int** input1, int** input2, int** input3, int** input4,char* filepath){
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));
    *input2 = ( int *)malloc(len * sizeof(int));
    *input3 = ( int *)malloc(len * sizeof(int));
    *input4 = ( int *)malloc(len * sizeof(int));



    int temp1;
    int temp2;
    int temp3;
    int temp4;
    while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4) {
        (*input1)[counter] = temp1;
        (*input2)[counter] = temp2;
        (*input3)[counter] = temp3;
        (*input4)[counter] = temp4;
        counter++;
    }

    fclose(fp);
    return len;
    
}

int main(int argc, char *argv[]) {
  // Variables
  int numNodePtrs;
  int numNodes;
  int *nodePtrs_h;
  int *nodeNeighbors_h;
  int *nodeVisited_h;
  int numTotalNeighbors_h;
  int *currLevelNodes_h;
  int numCurrLevelNodes;
  int numNextLevelNodes_h;
  int *nodeGate_h;
  int *nodeInput_h;
  int *nodeOutput_h;



  //output
  int *nextLevelNodes_h;



  numNodePtrs = read_input_one_two_four(&nodePtrs_h, "input1.raw");

  numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, "input2.raw");

  numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h,"input3.raw");

  numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, "input4.raw");
}


Writing read_input.c


### Write compareNodeOutput.c

In [ ]:
%%writefile compareNodeOutput.c

#include <stdio.h>
#include <stdlib.h>


void compareFiles(char *file_name1, char *file_name2) 
{ 
//get from https://www.tutorialspoint.com/c-program-to-compare-two-files-and-report-mismatches
FILE* fp1 = fopen(file_name1, "r");
FILE* fp2 = fopen(file_name2, "r");
    // fetching character of two file 
    // in two variable ch1 and ch2 
    char ch1 = getc(fp1); 
    char ch2 = getc(fp2); 
  
    // error keeps track of number of errors 
    // pos keeps track of position of errors 
    // line keeps track of error line 
    int error = 0, pos = 0, line = 1; 
  
    // iterate loop till end of file 
    while (ch1 != EOF && ch2 != EOF) 
    { 
        pos++; 
  
        // if both variable encounters new 
        // line then line variable is incremented 
        // and pos variable is set to 0 
        if (ch1 == '\n' && ch2 == '\n') 
        { 
            line++; 
            pos = 0; 
        } 
  
        // if fetched data is not equal then 
        // error is incremented 
        if (ch1 != ch2) 
        { 
            error++; 
            printf("Line Number : %d \tError"
               " Position : %d \n", line, pos); 
        } 
  
        // fetching character until end of file 
        ch1 = getc(fp1); 
        ch2 = getc(fp2); 
    } 
  
    printf("Total Errors : %d\t", error); 
} 

int main(int argc, char *argv[]){

    if( argc < 3) {
      printf("Require two files\n");
      exit(1);
      
   }
compareFiles(argv[1], argv[2]);
}


Writing compareNodeOutput.c


### Write compareNextLevelNodes.c

In [ ]:
%%writefile compareNextLevelNodes.c

#include <stdio.h>
#include <stdlib.h>
void sort(int *pointer, int size){
  //get from https://stackoverflow.com/questions/13012594/sorting-with-pointers-instead-of-indexes
    int *i, *j, temp;
    for(i = pointer; i < pointer + size; i++){
        for(j = i + 1; j < pointer + size; j++){
            if(*j < *i){
                temp = *j;
                *j = *i;
                *i = temp;
            }
        }
    }
}

void compareNextLevelNodeFiles(char *file_name1, char *file_name2) 
{ 

  
    FILE* fp_1 = fopen(file_name1, "r");
    if (fp_1 == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 

    FILE* fp_2 = fopen(file_name2, "r");
    if (fp_2 == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len_1;
    int len_2;
    int length_file_1 = fscanf(fp_1, "%d", &len_1);
    int length_file_2 = fscanf(fp_2, "%d", &len_2);

    if(length_file_1 != length_file_2){
      fprintf(stderr, "Wrong file length\n");
      exit(1);
    }
    int *input1 = (int *)malloc(len_1 * sizeof(int));
    int *input2 = (int *)malloc(len_2 * sizeof(int));




    int temp1;
    int temp2;

    while ((fscanf(fp_1, "%d", &temp1) == 1) && (fscanf(fp_2, "%d", &temp2) == 1)) {
        (input1)[counter] = temp1;
        (input2)[counter] = temp2;
        counter++;
    }

    sort(input1, len_1);
    sort(input2, len_2);

    for(int i=0; i< len_1; i++){
      if(input1[i] != input2[i]){
        fprintf(stderr, "Something goes wrong\n");
        exit(1);
      }
    }

    fprintf(stderr, "No errors!\n");
        exit(1);

} 

int main(int argc, char *argv[]){

    if( argc < 3) {
      printf("Require two files\n");
      exit(1);
      
   }
compareNextLevelNodeFiles(argv[1], argv[2]);
}


Writing compareNextLevelNodes.c


# Sequential

In [ ]:
%%writefile sequential.c

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

clock_t time_start;
clock_t time_end;

int gate_solver (int gate, int val1, int val2){
    
    int res;

    switch(gate) {
      case 0:
        res = val1 && val2;
        break;
      case 1:
        res = val1 || val2;
        break;
      case 2:
        res = !(val1 && val2);
        break;
      case 3:
        res = !(val1 || val2);
        break;
      case 4:
        res = val1 ^ val2;
        break;
      case 5:
        res = !(val1 ^ val2);
        break;
      default:
        printf("Invalid operator: %c\n", gate);
        break;
      }

    return res;
}

void sequential (int *nodePtrs_h, int *nodeNeighbors_h, int *nodeVisited_h,
                 int *currLevelNodes_h, int *nextLevelNodes_h,
                 int numCurrLevelNodes, int numNextLevelNodes_h,
                 int *nodeGate_h, int *nodeInput_h, int *nodeOutput_h)
{          

  for (int idx = 0; idx < numCurrLevelNodes; idx++){
      
      // # node's neighbors: nodeNeighbors[nodePtrs[node]] to nodeNeighbors[nodePtrs[node+1]]
      int node = currLevelNodes_h[idx];

      for (int nbrIdx = nodePtrs_h[node]; nbrIdx < nodePtrs_h[node + 1]; nbrIdx++){

          int neighbor = nodeNeighbors_h[nbrIdx];

          int nodeVisited = (nodeVisited_h[neighbor] && 1); 
          
          // # If the neighbor hasn't been visited
          if (!nodeVisited) {
              
              // # Set node to be visited
              nodeVisited_h[neighbor] = 1;

              // # Output results of gate operations
              nodeOutput_h[neighbor] = gate_solver(nodeGate_h[neighbor],
                                                 nodeOutput_h[node], 
                                                 nodeInput_h[neighbor]);
              // # nextLevelNodes output buffer
              nextLevelNodes_h[numNextLevelNodes_h] = neighbor;
              numNextLevelNodes_h++;
          }
      }
  }
}

// # Provided method from read_input.c
int read_input_one_two_four(int **input1, char* filepath){

 FILE* fp = fopen(filepath, "r");

    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));

    int temp1;

    while (fscanf(fp, "%d", &temp1) == 1) {
        (*input1)[counter] = temp1;

        counter++;
    }

    fclose(fp);
    return len;
}

// # Provided method from read_input.c
int read_input_three(int** input1, int** input2, int** input3, int** input4,char* filepath){
    
    FILE* fp = fopen(filepath, "r");
    if (fp == NULL){
     fprintf(stderr, "Couldn't open file for reading\n");
     exit(1);
    } 
    
    int counter = 0;
    int len;
    int length = fscanf(fp, "%d", &len);
    *input1 = ( int *)malloc(len * sizeof(int));
    *input2 = ( int *)malloc(len * sizeof(int));
    *input3 = ( int *)malloc(len * sizeof(int));
    *input4 = ( int *)malloc(len * sizeof(int));

    int temp1;
    int temp2;
    int temp3;
    int temp4;

    while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4) {
        (*input1)[counter] = temp1;
        (*input2)[counter] = temp2;
        (*input3)[counter] = temp3;
        (*input4)[counter] = temp4;
        counter++;
    }

    fclose(fp);
    return len;
}

int main(int argc, char *argv[]) {
    
  if (argc != 7)
  {
    printf("Usage: ./sequential <path_to_input_1.raw> <path_to_input_2.raw> <path_to_input_3.raw> <path_to_input_4.raw> <output_nodeOutput_filepath> <output_nextLevelNodes_filepath>\n");
    exit(1);
  }

  char *input_1 = argv[1];
  char *input_2 = argv[2];
  char *input_3 = argv[3];
  char *input_4 = argv[4];
  char *output1 = argv[5];
  char *output2 = argv[6];

  // # Variables from read_input.c
  int numNodePtrs;
  int numNodes;
  int *nodePtrs_h;
  int *nodeNeighbors_h;
  int *nodeVisited_h;
  int numTotalNeighbors_h;
  int *currLevelNodes_h;
  int numCurrLevelNodes;
  int numNextLevelNodes_h;
  int *nodeGate_h;
  int *nodeInput_h;
  int *nodeOutput_h;

  // # output from read_input.c
  int *nextLevelNodes_h;

  // # size of output buffer
  nextLevelNodes_h = ( int *)malloc(40101 * sizeof(int));

  numNodePtrs = read_input_one_two_four(&nodePtrs_h, input_1);

  numTotalNeighbors_h = read_input_one_two_four(&nodeNeighbors_h, input_2);

  numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, input_3);

  numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, input_4);

  time_start = clock();

  sequential(nodePtrs_h, nodeNeighbors_h, nodeVisited_h,
                 currLevelNodes_h, nextLevelNodes_h,
                 numCurrLevelNodes, numNextLevelNodes_h, 
             nodeGate_h, nodeInput_h, nodeOutput_h);

  time_end = clock();

  double time_spent = (double)(time_end - time_start) / CLOCKS_PER_SEC;
  printf("Time spent: %.3fms", time_spent * 1000);

  // # Write to output file nodeOutput.raw
  FILE* fp_output1 = fopen(output1, "w+");

  if (fp_output1 == NULL){
    printf("Output file failed to open");
    exit(EXIT_FAILURE);
  }

  // # Write to file number of element nodeOutput
  fprintf(fp_output1, "%d\n", 200000);

  // # Write to file elements of nodeOutput
  for (int i = 0; i < 200000; i++){
      fprintf(fp_output1, "%d\n", nodeOutput_h[i]);
  }

  fclose(fp_output1);

  // # Write to output file nextLevelNodes.raw
  FILE* fp_output2 = fopen(output2, "w+");

  if (fp_output2 == NULL){
    printf("Output file nextLevelNodes.raw failed to open");
    exit(EXIT_FAILURE);
  }
  
  // # Write to file number of element nextLevelNodes
  fprintf(fp_output2, "%d\n", 40101);

  // # Write to file elements of nextLevelNodes
  for(int i = 0; i < 40101; i++) {
      fprintf(fp_output2, "%d\n", nextLevelNodes_h[i]);
  }

  fclose(fp_output2);

}

Overwriting sequential.c


## Run sequential.c


In [ ]:
%%script bash
gcc sequential.c -o sequential

input1="input/input1.raw"
input2="input/input2.raw"
input3="input/input3.raw"
input4="input/input4.raw"
output1="output/nodeOutput.raw"
output2="output/nextLevelNodes.raw"

./sequential $input1 $input2 $input3 $input4 $output1 $output2

Time spent: 2.152ms

# Global Queuing

In [ ]:
%%writefile global_queuing.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

void *cudalloc(size_t len)
{
  void *p;

  int err = cudaMallocManaged(&p, len);
 
  if (err != 0)
  {
    printf("err: %d", err);
    return 0;
  }
 
  return p;
}

__device__ 
int gate_solver(int gate, int val1, int val2)
{
  int res;

  switch (gate)
  {
  case 0:
    res = val1 && val2;
    break;
  case 1:
    res = val1 || val2;
    break;
  case 2:
    res = !(val1 && val2);
    break;
  case 3:
    res = !(val1 || val2);
    break;
  case 4:
    res = val1 ^ val2;
    break;
  case 5:
    res = !(val1 ^ val2);
    break;
  default:
    printf("Invalid operator: %c\n", gate);
    break;
  }

  return res;
}

__global__ 
void process(int *nodePtrs_h, int *nodeNeighbors_h, int *nodeVisited_h,
             int *currLevelNodes_h, int *nextLevelNodes_h,
             int numCurrLevelNodes, int *numNextLevelNodes_h,
             int *nodeGate_h, int *nodeInput_h, int *nodeOutput_h, int elementsPerThread)
{
  int idx = blockIdx.x * blockDim.x + threadIdx.x;
  idx = idx * elementsPerThread;
 
  for (int j = 0; j < elementsPerThread; j++)
  {
    if (idx < numCurrLevelNodes) 
    {
      // # node's neighbors: nodeNeighbors[nodePtrs[node]] to nodeNeighbors[nodePtrs[node+1]]
      int node = currLevelNodes_h[idx];

      for (int nbrIdx = nodePtrs_h[node]; nbrIdx < nodePtrs_h[node + 1]; nbrIdx++)
      {
        int neighbor = nodeNeighbors_h[nbrIdx];
        int nodeVisited = atomicExch(&nodeVisited_h[neighbor], 1);

        // # If the neighbor hasn't been visited
        if (!nodeVisited)
        {
          // # Output results of gate operations
          nodeOutput_h[neighbor] = gate_solver(nodeGate_h[neighbor],
                                              nodeOutput_h[node],
                                              nodeInput_h[neighbor]);
         
          // # nextLevelNodes output buffer
          int write_position = atomicAdd(numNextLevelNodes_h, 1);
			  	nextLevelNodes_h[write_position] = neighbor;
        }
      }
    }
    idx++;
  }
}

// # Provided method from read_input.c
int read_input_one_two_four(int **input1, char *filepath)
{
  FILE *fp = fopen(filepath, "r");

  if (fp == NULL)
  {
    fprintf(stderr, "Couldn't open file for reading\n");
    exit(1);
  }

  int counter = 0;
  int len;
  int length = fscanf(fp, "%d", &len);
  *input1 = (int *)cudalloc(len * sizeof(int));

  int temp1;

  while (fscanf(fp, "%d", &temp1) == 1)
  {
    (*input1)[counter] = temp1;

    counter++;
  }

  fclose(fp);
  return len;
}

// # Provided method from read_input.c
int read_input_three(int **input1, int **input2, int **input3, int **input4, char *filepath)
{
  FILE *fp = fopen(filepath, "r");
  if (fp == NULL)
  {
    fprintf(stderr, "Couldn't open file for reading\n");
    exit(1);
  }

  int counter = 0;
  int len;
  int length = fscanf(fp, "%d", &len);
  *input1 = (int *)cudalloc(len * sizeof(int));
  *input2 = (int *)cudalloc(len * sizeof(int));
  *input3 = (int *)cudalloc(len * sizeof(int));
  *input4 = (int *)cudalloc(len * sizeof(int));

  int temp1;
  int temp2;
  int temp3;
  int temp4;

  while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4)
  {
    (*input1)[counter] = temp1;
    (*input2)[counter] = temp2;
    (*input3)[counter] = temp3;
    (*input4)[counter] = temp4;
    counter++;
  }

  fclose(fp);
  return len;
}

int main(int argc, char *argv[])
{
  if (argc != 7)
  {
    printf("Usage: ./global_queuing <path_to_input_1.raw> <path_to_input_2.raw> <path_to_input_3.raw> <path_to_input_4.raw> <output_nodeOutput_filepath> <output_nextLevelNodes_filepath>\n");
    exit(1);
  }

  char *input_1 = argv[1];
  char *input_2 = argv[2];
  char *input_3 = argv[3];
  char *input_4 = argv[4];
  char *output1 = argv[5];
  char *output2 = argv[6];

  // # Variables from read_input.c
  int numNodes;
  int *nodePtrs_h;
  int *nodeNeighbors_h;
  int *nodeVisited_h;
  int *currLevelNodes_h;
  int numCurrLevelNodes;
  int *nodeGate_h;
  int *nodeInput_h;
  int *nodeOutput_h;
 
  int *numNextLevelNodes_h = (int*)cudalloc(sizeof(int));
  *numNextLevelNodes_h = 0;

  // # output from read_input.c
  int *nextLevelNodes_h;
 
  // # size of output buffer
  nextLevelNodes_h = (int *)cudalloc(40101 * sizeof(int));
 
	// # Launch kernel with unique stream ID
	cudaSetDevice(0); 

  read_input_one_two_four(&nodePtrs_h, input_1);
  read_input_one_two_four(&nodeNeighbors_h, input_2);
  numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, input_3);
  numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, input_4);

  int numBlock = 10;     // # 10, 25, 35
  int blockSize = 32;    // # 32, 64, 128
 
  /*
    # Used to plot data
    int numBlock = atoi(argv[7]);
    int blockSize = atoi(argv[8]);
  */
 
  int elementsPerThread = numCurrLevelNodes / (numBlock * blockSize);
 
  // # if division != 0, add another element to thread
  if(numCurrLevelNodes % (numBlock * blockSize) != 0){
    elementsPerThread++;
  }

  // # Timer starts
	float time;
	cudaEvent_t start, stop;

	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
 
  process<<<numBlock, blockSize>>>(nodePtrs_h, nodeNeighbors_h, nodeVisited_h,
                                  currLevelNodes_h, nextLevelNodes_h,
                                  numCurrLevelNodes, numNextLevelNodes_h,
                                  nodeGate_h, nodeInput_h, nodeOutput_h, elementsPerThread);

  cudaDeviceSynchronize();
 
 	// # Timer stops
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&time, start, stop);
	printf("(%d numBlock, %d blockSize): %3.3f ms\n", numBlock, blockSize, time);

  // # Write to output file nodeOutput.raw
  FILE *fp_output1 = fopen(output1, "w+");

  if (fp_output1 == NULL)
  {
    printf("Output file failed to open");
    exit(EXIT_FAILURE);
  }

  // # Write to file number of element nodeOutput
  fprintf(fp_output1, "%d\n", numNodes);

  // # Write to file elements of nodeOutput
  for (int i = 0; i < numNodes; i++)
  {
    fprintf(fp_output1, "%d\n", nodeOutput_h[i]);
  }

  fclose(fp_output1);

  // # Write to output file nextLevelNodes.raw
  FILE *fp_output2 = fopen(output2, "w+");

  if (fp_output2 == NULL)
  {
    printf("Output file nextLevelNodes.raw failed to open");
    exit(EXIT_FAILURE);
  }

  // # Write to file number of element nextLevelNodes
  fprintf(fp_output2, "%d\n", 40101);

  // # Write to file elements of nextLevelNodes
  for (int i = 0; i < 40101; i++)
  {
    fprintf(fp_output2, "%d\n", nextLevelNodes_h[i]);
  }

  fclose(fp_output2);
}

Writing global_queuing.cu


## Run global_queuing.cu


In [ ]:
%%script bash
nvcc global_queuing.cu -o global_queuing

input1="input/input1.raw"
input2="input/input2.raw"
input3="input/input3.raw"
input4="input/input4.raw"
output1="output/GQ-nodeOutput.raw"
output2="output/GQ-nextLevelNodes.raw"

./global_queuing $input1 $input2 $input3 $input4 $output1 $output2

# --- FOR PLOT---
# ./global_queuing $input1 $input2 $input3 $input4 $output1 $output2 10 32
# ./global_queuing $input1 $input2 $input3 $input4 $output1 $output2 10 64
# ./global_queuing $input1 $input2 $input3 $input4 $output1 $output2 10 128
# ./global_queuing $input1 $input2 $input3 $input4 $output1 $output2 25 32
# ./global_queuing $input1 $input2 $input3 $input4 $output1 $output2 25 64
# ./global_queuing $input1 $input2 $input3 $input4 $output1 $output2 25 128
# ./global_queuing $input1 $input2 $input3 $input4 $output1 $output2 35 32
# ./global_queuing $input1 $input2 $input3 $input4 $output1 $output2 35 64
# ./global_queuing $input1 $input2 $input3 $input4 $output1 $output2 35 128

(10 numBlock, 32 blockSize): 1.657 ms


# Block Queuing

In [ ]:
%%writefile block_queuing.cu

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

#include "cuda_runtime.h"
#include "device_launch_parameters.h"

void *cudalloc(size_t len)
{
  void *p;

  int err = cudaMallocManaged(&p, len);
 
  if (err != 0)
  {
    printf("err: %d", err);
    return 0;
  }
 
  return p;
}

__device__ 
int gate_solver(int gate, int val1, int val2)
{
  int res;

  switch (gate)
  {
  case 0:
    res = val1 && val2;
    break;
  case 1:
    res = val1 || val2;
    break;
  case 2:
    res = !(val1 && val2);
    break;
  case 3:
    res = !(val1 || val2);
    break;
  case 4:
    res = val1 ^ val2;
    break;
  case 5:
    res = !(val1 ^ val2);
    break;
  default:
    printf("Invalid operator: %c\n", gate);
    break;
  }

  return res;
}

__global__ 
void process(int *nodePtrs_h, int *nodeNeighbors_h, int *nodeVisited_h,
             int *currLevelNodes_h, int *globalQueue,
             int numCurrLevelNodes, int *numNextLevelNodes_h,
             int *nodeGate_h, int *nodeInput_h, int *nodeOutput_h,
             int elementsPerThread, int sharedQueueSize)
{
  int idx = (blockIdx.x * blockDim.x + threadIdx.x) * elementsPerThread;
 
  // Initialize shared queue
  __shared__ int blockQueue64[64];
  __shared__ int blockQueue32[32];
  __shared__ int *blockQueue;
  __shared__ int queueCurrentSize;

  // # Initialize shared index only on first thread
  if (threadIdx.x == 0)
  {
    queueCurrentSize = 0;

    if(sharedQueueSize == 32) {
      blockQueue = blockQueue32;
    } else if (sharedQueueSize == 64){
      blockQueue = blockQueue64;
    } else {
      printf("Invalid block queue size, only 32 and 64 supported.\n");
    }
   
    queueCurrentSize = 0;
  }

  // # Wait and start all threads at the same time
  __syncthreads();

  for (int j = 0; j < elementsPerThread; j++)
  {
    if (idx < numCurrLevelNodes)
    {
      // # node's neighbors: nodeNeighbors[nodePtrs[node]] to nodeNeighbors[nodePtrs[node+1]]
      int node = currLevelNodes_h[idx];

      for (int nbrIdx = nodePtrs_h[node]; nbrIdx < nodePtrs_h[node + 1]; nbrIdx++)
      {
        int neighbor = nodeNeighbors_h[nbrIdx];
        int nodeVisited = atomicExch(&nodeVisited_h[neighbor], 1);

        // # If the neighbor hasn't been visited
        if (!nodeVisited)
        {

          // # Output results of gate operations
          nodeOutput_h[neighbor] = gate_solver(nodeGate_h[neighbor],
                                               nodeOutput_h[node],
                                               nodeInput_h[neighbor]);

          int write_position = atomicAdd(&queueCurrentSize, 1);

          // # Write in block queue when we have room
          if (write_position < sharedQueueSize)
          {
            blockQueue[write_position] = neighbor;
          }
          // # Add directly to global queue when block queue is full
          else
          {
            // # We didn't write to local queue, so we decrement the index
            atomicSub(&queueCurrentSize, 1);
             
            int write_position = atomicAdd(numNextLevelNodes_h, 1);
            globalQueue[write_position] = neighbor;
          }
        }
      }
    }
    idx++;
  }

  // Wait for other threads to finish
  __syncthreads();

  if (threadIdx.x == 0)
  {
    // # Copy block queue into global queue
    int globalIndex = atomicAdd(numNextLevelNodes_h, queueCurrentSize);
    for (int i = 0; i < queueCurrentSize; ++i)
    {
      globalQueue[globalIndex + i] = blockQueue[i];
    }
  }
}

// # Provided method from read_input.c
int read_input_one_two_four(int **input1, char *filepath)
{
  FILE *fp = fopen(filepath, "r");

  if (fp == NULL)
  {
    fprintf(stderr, "Couldn't open file for reading\n");
    exit(1);
  }

  int counter = 0;
  int len;
  int length = fscanf(fp, "%d", &len);
  *input1 = (int *)cudalloc(len * sizeof(int));

  int temp1;

  while (fscanf(fp, "%d", &temp1) == 1)
  {
    (*input1)[counter] = temp1;

    counter++;
  }

  fclose(fp);
  return len;
}

// # Provided method from read_input.c
int read_input_three(int **input1, int **input2, int **input3, int **input4, char *filepath)
{
  FILE *fp = fopen(filepath, "r");
  if (fp == NULL)
  {
    fprintf(stderr, "Couldn't open file for reading\n");
    exit(1);
  }

  int counter = 0;
  int len;
  int length = fscanf(fp, "%d", &len);
  *input1 = (int *)cudalloc(len * sizeof(int));
  *input2 = (int *)cudalloc(len * sizeof(int));
  *input3 = (int *)cudalloc(len * sizeof(int));
  *input4 = (int *)cudalloc(len * sizeof(int));

  int temp1;
  int temp2;
  int temp3;
  int temp4;

  while (fscanf(fp, "%d,%d,%d,%d", &temp1, &temp2, &temp3, &temp4) == 4)
  {
    (*input1)[counter] = temp1;
    (*input2)[counter] = temp2;
    (*input3)[counter] = temp3;
    (*input4)[counter] = temp4;
    counter++;
  }

  fclose(fp);
  return len;
}

int main(int argc, char *argv[])
{
  if (argc != 10)
  {
    printf("Usage: ./global_queuing <numBlock> <blockSize> <sharedQueueSize> <path_to_input_1.raw> <path_to_input_2.raw> <path_to_input_3.raw> <path_to_input_4.raw> <output_nodeOutput_filepath> <output_nextLevelNodes_filepath>\n");
    exit(1);
  }
 
  int numBlock = atoi(argv[1]);
  int blockSize = atoi(argv[2]);
  int sharedQueueSize = atoi(argv[3]);
  char *input_1 = argv[4];
  char *input_2 = argv[5];
  char *input_3 = argv[6];
  char *input_4 = argv[7];
  char *output1 = argv[8];
  char *output2 = argv[9];

  // # Variables from read_input.c
  int numNodes;
  int *nodePtrs_h;
  int *nodeNeighbors_h;
  int *nodeVisited_h;
  int *currLevelNodes_h;
  int numCurrLevelNodes;
  int *nodeGate_h;
  int *nodeInput_h;
  int *nodeOutput_h;
 
  int *numNextLevelNodes_h = (int*)cudalloc(sizeof(int));
  *numNextLevelNodes_h = 0;

  // # output from read_input.c
  int *nextLevelNodes_h;
 
  // # size of output buffer
  nextLevelNodes_h = (int *)cudalloc(40101 * sizeof(int));
 
	// # Launch kernel with unique stream ID
	cudaSetDevice(0); 

  read_input_one_two_four(&nodePtrs_h, input_1);
  read_input_one_two_four(&nodeNeighbors_h, input_2);
  numNodes = read_input_three(&nodeVisited_h, &nodeGate_h, &nodeInput_h, &nodeOutput_h, input_3);
  numCurrLevelNodes = read_input_one_two_four(&currLevelNodes_h, input_4);
 
  int elementsPerThread = numCurrLevelNodes / (numBlock * blockSize);
 
  // # if division != 0, add another element to thread
  if(numCurrLevelNodes % (numBlock * blockSize) != 0){
    elementsPerThread++;
  }

  // # Timer starts
	float time;
	cudaEvent_t start, stop;

	cudaEventCreate(&start);
	cudaEventCreate(&stop);
	cudaEventRecord(start, 0);
 
  process<<<numBlock, blockSize >>>(nodePtrs_h, nodeNeighbors_h, nodeVisited_h,
                                  currLevelNodes_h, nextLevelNodes_h,
                                  numCurrLevelNodes, numNextLevelNodes_h,
                                  nodeGate_h, nodeInput_h, nodeOutput_h, elementsPerThread,
                                  sharedQueueSize);

  cudaDeviceSynchronize();
 
 	// # Timer stops
	cudaEventRecord(stop, 0);
	cudaEventSynchronize(stop);
	cudaEventElapsedTime(&time, start, stop);
	printf("(%d numBlock, %d blockSize): %3.3f ms\n", numBlock, blockSize, time);

  // # Write to output file nodeOutput.raw
  FILE *fp_output1 = fopen(output1, "w+");

  if (fp_output1 == NULL)
  {
    printf("Output file failed to open");
    exit(EXIT_FAILURE);
  }

  // # Write to file number of element nodeOutput
  fprintf(fp_output1, "%d\n", numNodes);

  // # Write to file elements of nodeOutput
  for (int i = 0; i < numNodes; i++)
  {
    fprintf(fp_output1, "%d\n", nodeOutput_h[i]);
  }

  fclose(fp_output1);

  // # Write to output file nextLevelNodes.raw
  FILE *fp_output2 = fopen(output2, "w+");

  if (fp_output2 == NULL)
  {
    printf("Output file nextLevelNodes.raw failed to open");
    exit(EXIT_FAILURE);
  }

  // # Write to file number of element nextLevelNodes
  fprintf(fp_output2, "%d\n", 40101);

  // # Write to file elements of nextLevelNodes
  for (int i = 0; i < 40101; i++)
  {
    fprintf(fp_output2, "%d\n", nextLevelNodes_h[i]);
  }

  fclose(fp_output2);
}

Writing block_queuing.cu


## Run block_queuing.cu


In [ ]:
%%script bash
nvcc block_queuing.cu -o global_queuing

input1="input/input1.raw"
input2="input/input2.raw"
input3="input/input3.raw"
input4="input/input4.raw"
output1="output/BQ-nodeOutput.raw"
output2="output/BQ-nextLevelNodes.raw"

./global_queuing 32 25 32 $input1 $input2 $input3 $input4 $output1 $output2
./global_queuing 32 25 64 $input1 $input2 $input3 $input4 $output1 $output2
./global_queuing 32 35 32 $input1 $input2 $input3 $input4 $output1 $output2
./global_queuing 32 35 64 $input1 $input2 $input3 $input4 $output1 $output2

./global_queuing 64 25 32 $input1 $input2 $input3 $input4 $output1 $output2
./global_queuing 64 25 64 $input1 $input2 $input3 $input4 $output1 $output2
./global_queuing 64 35 32 $input1 $input2 $input3 $input4 $output1 $output2
./global_queuing 64 35 64 $input1 $input2 $input3 $input4 $output1 $output2

(32 numBlock, 25 blockSize): 1.839 ms
(32 numBlock, 25 blockSize): 1.897 ms
(32 numBlock, 35 blockSize): 1.854 ms
(32 numBlock, 35 blockSize): 1.880 ms
(64 numBlock, 25 blockSize): 1.962 ms
(64 numBlock, 25 blockSize): 1.864 ms
(64 numBlock, 35 blockSize): 1.790 ms
(64 numBlock, 35 blockSize): 1.534 ms


# Test equality

In [ ]:
%%script bash
gcc compareNextLevelNodes.c -o compareNextLevelNodes

actual="output/BQ-nextLevelNodes.raw"

./compareNextLevelNodes input/sol_nextLevelNodes.raw $actual

No errors!


In [ ]:
%%script bash
gcc compareNodeOutput.c -o compareNodeOutput

actual="output/BQ-nodeOutput.raw"

./compareNodeOutput input/sol_nodeOutput.raw $actual

Total Errors : 0	